# Example script for reconstructing 3D data sets

3D CT reconstruction using the FBP algorithm from data stored in a .npz file 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import iradon
import svmbir 

In [ ]:
#Path to file that contains the sinograms and angles 
data_file = 'orhs_apr21_ex_new.npz'

In [ ]:
#Read .npz file 
sinogram = np.load(data_file)['sinogram']
angles = np.load(data_file)['angles']

In [ ]:
#Print sinogram and angles shape to verify data 
print(sinogram.shape)
print(angles.shape)

#Sinogram is stored in the order of slice, column index, angle index 

In [ ]:
#Sub-set data to reduce computation (for demo) 
NUM_SLICE_RECON=4
VIEW_SUBSAMP=4
sinogram = sinogram[256:256+NUM_SLICE_RECON,:,::VIEW_SUBSAMP]
angles = angles[::VIEW_SUBSAMP]

In [ ]:
#Plot 
plt.figure();plt.imshow(sinogram[0]);plt.colorbar()
plt.figure();plt.imshow(sinogram[NUM_SLICE_RECON-1]);plt.colorbar()
plt.figure();plt.plot(angles)

#Another look at the sinogram array 
plt.figure();plt.imshow(np.squeeze(sinogram[:,:,0]));plt.colorbar()
plt.figure();plt.imshow(np.squeeze(sinogram[:,:,sinogram.shape[2]-1]));plt.colorbar()

In [ ]:
#FBP reconstruction for each sinogram 
num_slice = sinogram.shape[0]
num_col = sinogram.shape[1]
recon_fbp = np.zeros((num_slice,num_col,num_col)).astype(np.float32)
for idx in range(num_slice):
    print('Reconstructing slice %d of %d'%(idx,num_slice))
    recon_fbp[idx]=iradon(sinogram[idx],angles,circle=True,filter='ramp')

In [ ]:
#MBIR reconstruction
#Sinogram has to be of order of angles, slice index, column index 
#Angles have to be in radians 
MRF_P=1.2
sharpness = 0.0
T=0.1
snr_db = 40.0
angles_rad = angles*np.pi/180
recon_mbir = svmbir.recon(sinogram.swapaxes(0,2).swapaxes(1,2), angles_rad, p=MRF_P,T = T,snr_db=snr_db,positivity=False,verbose=1)

In [ ]:
#Plot a reconstructed slice
disp_slice_idx = num_slice-1
DISP_MIN=-0.002
DISP_MAX=0.006
plt.figure();plt.imshow(recon_fbp[disp_slice_idx],cmap='gray',vmin=DISP_MIN,vmax=DISP_MAX);plt.colorbar()
plt.figure();plt.imshow(np.flipud(np.rot90(recon_mbir[disp_slice_idx])),cmap='gray',vmin=DISP_MIN,vmax=DISP_MAX);plt.colorbar()

In [ ]:
#Write to tiff files 
import dxchange
dxchange.writer.write_tiff_stack(recon_fbp, fname='test_recon/fbp_slice.tiff', axis=0, digit=5, start=0, overwrite=False) 
dxchange.writer.write_tiff_stack(recon_mbir, fname='test_recon/mbir_slice.tiff', axis=0, digit=5, start=0, overwrite=False) 